In [ ]:
import os
import sys
import json
import requests
import re
from docx import Document
from google.genai import types
import mimetypes
import base64

my_var = os.getenv("GEMINI_API_KEY")
print(my_var)
print(os.environ)

# 🔹 Constants
COMPANY_ID = "de86535f-7695-4aa4-9654-78906191298a"
LOGIN_URL = "https://v1.cclhp.eu/api/users/login"
QUERY_URL = f"https://v1.cclhp.eu/api/datasets/"
EMAIL = "debug@debug.debug"
PASSWORD = "debug@debug.debug"



In [86]:
def read_docx(file_path):
    doc = Document(file_path)
    return "\n".join([p.text for p in doc.paragraphs if p.text.strip()])

# 🔹 Function to get JWT token
def get_jwt_token():
    payload = {"email": EMAIL, "password": PASSWORD}
    headers = {"Content-Type": "application/json", "Accept": "application/json"}

    response = requests.post(LOGIN_URL, json=payload, headers=headers)
    
    if response.status_code == 200:
        print(f"Command Center is Online✅")
        return response.json()["proxy_response"]["jwt_token"]
    else:
        print(f"❌ Failed to authenticate: {response.status_code}, {response.text}")
        sys.exit(1)

def extract_json(client,chat, response, jwt_token):
    # Extract the JSON block
    match = re.search(r"```json\s*(\{.*?\})\s*```", response.text, re.DOTALL)
    if match:
        json_str = match.group(1)
        data = json.loads(json_str)
        print("Extracted JSON:", data)
            
        # Route logic based on relevance
        if data.get("relevance", "").lower() == "yes":
            handle_bi_relevant(client, chat, data, jwt_token)
        else:
            handle_non_bi_query(data)
    else:
        print("No JSON found in the response.")
        

# 🔹 Function to upload CSV file
def extract_datasets(jwt_token):
    headers = {
        "CCapi-company-id": COMPANY_ID,
        "Authorization": f"Bearer {jwt_token}",
        "Accept": "application/json"
    }

    response = requests.get(QUERY_URL, headers=headers)
    
    if response.status_code == 200:
        print(f"✅ Successfully Extracted Datasets!")
        print(response.text,response.status_code)
        return response.text
    else:
        print(f"❌ Extraction failed: {response.status_code}, {response.text}")

def handle_bi_relevant(client, chat, data, jwt_token):
    # Route logic based on data presence
    if data.get("data_presence", "").lower() == "yes":
        handle_fully_relevant(client, chat, data,jwt_token)
    else:
        handle_partially_relevant(client,chat, data,jwt_token)
    
def handle_non_bi_query(data):
    print(f"Gemini: {data.get("answer", "")}")
    #restart_chat()
    
def handle_fully_relevant(client, chat,data,jwt_token):
    print(f"Command Center Agent is connected via Gemini")
    datasets = extract_datasets(jwt_token)
    # Send first message with system instruction + actual user query
    variable = data.get("variable", "")
    chart_data = {
        "dataset_id": "your_dataset_id",
        "title": "My Chart Title",
        "chart_type": "line",
        "chart_configuration": {
            "config_type": "line_chart_config",
            "background_colors": ["#ffffff"],
            "border_colors": ["#000000"],
            "dimension_field": "date",
            "dimension_aggregation": "none",
            "metrics": [
                {
                    "metric_field": "revenue",
                    "metric_aggregation": "sum"
                }
            ],
            "thresholds": [
                {
                    "metric_field": "revenue",
                    "label": "Target",
                    "value": 10000,
                    "color": "red",
                    "conditional_formatting": {
                        "direction": "above",
                        "tolerance": 500
                    }
                }
            ],
            "breakdown": "region"
        },
        "tags": ["finance", "monthly", "performance"],
        "preferences": {}
    }

    json_format = {
        "call_parameters": {
            "date_from": "2025-04-17T00:00:00.000Z",
            "date_to": "2025-04-17T23:59:59.000Z",
            "page": None,
            "sort_field": None,
            "sort_order": None,
            "filters": None
        },
        "headers": {
            "CCapi-company-id": "de86535f-7695-4aa4-9654-78906191298a"
        },
        "request_payload": {
            "dataset_id": "380a05d2-c3d8-498e-9b32-ebbd4d76561f",
            "title": "Energy Consumption - Apparent Power",
            "chart_type": "line",
            "chart_configuration": {
                "config_type": "line_chart_config",
                "background_colors": ["#ffffff"],
                "border_colors": ["#000000"],
                "dimension_field": "timestamp_sensor",
                "dimension_aggregation": "hours",
                "metrics": [
                    {
                        "metric_field": "apparent_power",
                        "metric_aggregation": "sum"
                    }
                ],
                "thresholds": [],
                "breakdown": None
            },
            "preferences": {
                "multipleScale": False
            },
            "tags": []
        }
    }

    file_path = "API-DOCS (The lost documentation).docx"

    docx_content = read_docx(file_path)

    system_instruction = (
        f"Now that you have extracted the relevant variable name for BI relevant queries, I want you to compare it with the datsets present in the BI tool "
        f" The variable name is {variable}."
        f"The datasets are provided in the end "
        f"If the variable is either directly present in a given dataset or can be represented as other variables in a dataset visualized together, I need you to create a json which is to be exported to the BI tool to extract relevant data and chart configuration "
        f"organize the json in three entities: call parameters, headers and request payload"
        f"the documentation reference for the call parameters, header and payload configuration is as follows"
        f"{docx_content}"
        f"The example format for payload is as follows"
        f"{json.dumps(chart_data, indent=2)}"
        f" final output json format is as follows"
        f"{json_format}"
        #f"If the variable is either directly present or can be calculated using multiple variables present in one dataset, create a json with if the dataset ahs been found as boolean and the name of the dataset "t
        f"if the variable is not directly present or can't be represents using other variables present in one of the datasets, inform the user the same"
    )

    response = chat.send_message(system_instruction + datasets)

    chart_params = extract_data(response)

    if chart_params is None:
        print("Handle missing JSON case here.")
    else:
        request_cc(client, chat,chart_params, jwt_token)

    # Print chat
    print("Assistant:", response.text)
    print(chart_params)

def handle_partially_relevant(client,chat,data,jwt_token):
    print("Assistant:", data.get("missing_information", "") )
    user_query = input("User Input: ")
    system_instruction = (
        "Check if the data provided by the user attached below along with the previous user input is now fully sufficient for BI tool input"
        "If the full data is present accumulate the data using the same Json format mentioned before (relevance, data presence, etc). Same situation if the data is not fully present"
    )    
    
    response = chat.send_message(system_instruction + user_query)

    print("Assistant:", response.text)

    extract_json(client,chat,response,jwt_token)


def extract_data(response):
# Extract the JSON block
    match = re.search(r"```json\s*(\{.*?\})\s*```", response.text, re.DOTALL)
    if match:
        json_str = match.group(1)
        data = json.loads(json_str)
        print("Extracted JSON:", data)
        return data
                
    else:
        print("No JSON found in the response.")
        return None
        
def request_cc(client, chat, input_data, jwt_token):
    base_url = "https://v1.cclhp.eu/api/charts/preview"
    params = input_data["call_parameters"]
    query_params = {
        "date_from": params["date_from"],
        "date_to": params["date_to"]
    }
    # Add optional parameters if they are not None
    if params.get("page") is not None:
        query_params["page"] = params["page"]
    if params.get("sort_field") is not None:
        query_params["sort_field"] = params["sort_field"]
    if params.get("sort_order") is not None:
        query_params["sort_order"] = params["sort_order"]

    
    # Final URL with query string
    response = requests.post(
        url=base_url,
        headers={
            "Content-Type": "application/json",
            "Authorization": f"Bearer {jwt_token}",
            "CCapi-company-id": input_data["headers"]["CCapi-company-id"]
        },
        params=query_params,
        json=input_data["request_payload"]
    )

    # Print results
    print("Status Code:", response.status_code)
    try:
        print("Response JSON:", response.json())
        create_chart(client, chat, response.json)
    except json.decoder.JSONDecodeError:
        print("Response Text:", response.text)
        
def create_chart(client, chat, input):
    system_instruction = (
    f"I would like you to generate the chart by yourself by writing your own code and exporting the final result as image to me "
    f"The response (chart input) are as follows:"
    f"{input}"
    f"focus on the metric field datapoints for chart creation"
    f"along with the chart image I would also request you to provide some text resonse about what could be some genereal statistics using the data points for the metric field keeping in mind what the user had initially requested"
    f"just use the response provided to create the chart, dont show me the steps, elaborate or ask for additional input. All I need is the chart image and the statistics as text as response"
    )

    print(system_instruction)

    generate(client,system_instruction)

    #response = chat.send_message(system_instruction)

    #print(response)

def save_binary_file(file_name, data):
    f = open(file_name, "wb")
    f.write(data)
    f.close()

def generate(client, input):

    model = "gemini-2.0-flash-exp-image-generation"
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text= input),
            ],
        ),
    ]
    generate_content_config = types.GenerateContentConfig(
        response_modalities=[
            "image",
            "text",
        ],
        response_mime_type="text/plain",
    )

    for chunk in client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    ):
        if chunk.candidates is None or chunk.candidates[0].content is None or chunk.candidates[0].content.parts is None:
            continue
        if chunk.candidates[0].content.parts[0].inline_data:
            file_name = "chart"
            inline_data = chunk.candidates[0].content.parts[0].inline_data
            file_extension = mimetypes.guess_extension(inline_data.mime_type)
            save_binary_file(
                f"{file_name}{file_extension}", inline_data.data
            )
            print(
                "File of mime type"
                f" {inline_data.mime_type} saved"
                f"to: {file_name}"
            )
        else:
            print(chunk.text)



    

    


In [87]:
from google import genai

# Initialize client
client = genai.Client(api_key=my_var)

# Create a chat (no system_instruction passed here)
chat = client.chats.create(model="gemini-2.0-flash")

try:
    # Get JWT token
    jwt_token = get_jwt_token()


except Exception as e:
    print(json.dumps({"error": str(e)}))


print(f"**************Command Center Agent***********************")




Command Center is Online✅
**************Command Center Agent***********************


In [88]:
# Send first message with system instruction + actual user query
system_instruction = (
    "You are an interface for an agent connected to a BI Tool called Command Center. "
    "First, check if the user's question is relevant for a BI tool. "
    "If it's not, create a json with relevance and the then just answer it as best you can in the json frame. "
    "If it is relevant, answer yes and then check if the necessary data (like timeframe and BI variable) is provided. "
    "If all data is present, create a JSON object with the data presence as yes and with the variable name, time start in 24hr format, and time end in 24hr format. else create a json similar json as before but with data presence as no and variable also asking the user to input more information \n\n"
)

#user_question = "How many users signed up between 9:00 and 17:00 yesterday?"
#user_question = "who is the pope?"
#user_question = "whats the production? capacity"
user_question = input("User Input: ")



response = chat.send_message(system_instruction + user_question)

# Print chat
print("User:", user_question)
print("Assistant:", response.text)

extract_json(client,chat,response, jwt_token)



User: what is the average hourly power consumption today?
Assistant: ```json
{
  "relevance": "yes",
  "data_presence": "yes",
  "variable": "power consumption",
  "time_start": "00:00",
  "time_end": "23:59"
}
```

Extracted JSON: {'relevance': 'yes', 'data_presence': 'yes', 'variable': 'power consumption', 'time_start': '00:00', 'time_end': '23:59'}
Command Center Agent is connected via Gemini
✅ Successfully Extracted Datasets!
[{"name":"sensor","description":"sensor","data_schema":{"type":"object","properties":{"timestamp_sensor":{"type":"string","cc_def":{"type":"timestamp"}},"tension_l1_n":{"type":"number","cc_def":{"type":"numerical","scope":"continuous"}},"tension_l2_n":{"type":"number","cc_def":{"type":"numerical","scope":"continuous"}},"tension_l3_n":{"type":"number","cc_def":{"type":"numerical","scope":"continuous"}},"apparent_power":{"type":"number","cc_def":{"type":"numerical","scope":"continuous"}}}},"timestamp_field":"timestamp_sensor","tags":null,"auditable":{"created_by

ServerError: 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.', 'status': 'UNAVAILABLE'}}

In [15]:
print(response)

candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='```json\n{\n  "relevance": "yes",\n  "data_presence": "no",\n  "variable_name": "production",\n  "missing_information": "Please specify the timeframe for the production data you are interested in (e.g., \'last week\', \'today\', \'this month\')."\n}\n```\n')], role='model'), citation_metadata=None, finish_message=None, token_count=None, avg_logprobs=-0.10746549271248482, finish_reason=<FinishReason.STOP: 'STOP'>, grounding_metadata=None, index=None, logprobs_result=None, safety_ratings=None)] create_time=None response_id=None model_version='gemini-2.0-flash' prompt_feedback=None usage_metadata=GenerateContentResponseUsageMetadata(cached_content_token_count=None, candidates_token_count=74, prompt_token_count=153, total_token_count=227) automatic_function_calling_history=[

In [ ]:



# Extract the JSON block
match = re.search(r"```json\s*(\{.*?\})\s*```", response.text, re.DOTALL)
if match:
    json_str = match.group(1)
    data = json.loads(json_str)
    print("Extracted JSON:", data)
        
    # Route logic based on relevance
    if data.get("relevance", "").lower() == "yes":
        handle_bi_relevant(data)
    else:
        handle_non_bi_query(data)
else:
    print("No JSON found in the response.")

Extracted JSON: {'relevance': 'yes', 'data_presence': 'no', 'variable_name': 'production/production capacity', 'missing_information': "Please specify which you're interested in - production or production capacity - and provide a timeframe (e.g., last week, last month, specific dates)."}


In [22]:
variable = "abc"
system_instruction = (
    f"Now that you have extracted the relevant variable name for BI relevant queries, I want you to compare it with the datsets present in the BI tool "
    f" The variable name is {variable}."
    f"The datasets are provided in the end "
    f"If the variable is either directly present or can be calculated using multiple variables present in one dataset, create a json with if the dataset ahs been found as boolean and the name of the dataset "
    )
print(system_instruction)

Now that you have extracted the relevant variable name for BI relevant queries, I want you to compare it with the datsets present in the BI tool  The variable name is abc.The datasets are provided in the end If the variable is either directly present or can be calculated using multiple variables present in one dataset, create a json with if the dataset ahs been found as boolean and the name of the dataset 


In [4]:
response = client.models.generate_content_stream(
    model="gemini-2.0-flash",
    contents=["Explain how AI works"]
)
for chunk in response:
    print(chunk.text, end="")

Okay, let's break down how AI works in a way that's understandable, even if you don't have a computer science background.  I'll try to avoid getting too technical, but I'll also give you some core concepts.

**The Basic Idea: Mimicking Intelligence**

At its heart, Artificial Intelligence (AI) is about creating computer systems that can perform tasks that typically require human intelligence. This includes things like:

*   **Learning:**  Improving performance over time based on data.
*   **Reasoning:**  Drawing conclusions from information.
*   **Problem Solving:** Finding solutions to complex issues.
*   **Perception:** Understanding sensory input (like images, sound, or text).
*   **Natural Language Processing:** Understanding and generating human language.

**Key Components & Approaches**

Think of AI as an umbrella term covering a lot of different techniques. Here are some of the most important:

1.  **Machine Learning (ML): The Learning Engine**

    *   **What it is:** Machine l

In [5]:
chat = client.chats.create(model="gemini-2.0-flash")

response = chat.send_message("I have 2 dogs in my house.")
print(response.text)



That's wonderful! Dogs bring so much joy to a home. Do you want to tell me anything else about them? Like their names, breeds, or favorite activities?



In [6]:
response = chat.send_message("How many paws are in my house?")
print(response.text)

for message in chat.get_history():
    print(f'role - {message.role}',end=": ")
    print(message.parts[0].text)

Since you have two dogs, and each dog has four paws, there are a total of 8 paws in your house.

role - user: I have 2 dogs in my house.
role - model: That's wonderful! Dogs bring so much joy to a home. Do you want to tell me anything else about them? Like their names, breeds, or favorite activities?

role - user: How many paws are in my house?
role - model: Since you have two dogs, and each dog has four paws, there are a total of 8 paws in your house.

